In [3]:
from datasets import load_dataset
dataset = load_dataset('wikipedia', "20220301.en")

/Users/g.frigo/.pyenv/versions/3.11.1/envs/relu/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

In [93]:
import mmap
import os
with open("test.db", mode="a+", encoding="utf-8") as file_obj:
    os.ftruncate(file_obj.fileno(), 2**30)

#    with mmap.mmap(file_obj.fileno(), length=0, access=mmap.ACCESS_WRITE) as mmap_obj:
#        mmap_obj.write(bytes("".join([chr(i) for i in range(256)]),"utf-8"))
#        mmap_obj.flush()
#        print(mmap_obj.size())

#def billy(s:mmap,pagesize)

In [84]:
from typing import TextIO
def read_chunk(file:TextIO,chunk_size:int=1024)->str:
    while True:
        str_chunk = file.read(chunk_size)
        if not str_chunk:
            break
        yield str_chunk

char_frequency={}
with open("resources/shakespeare.txt", mode="r", encoding="utf-8") as file_obj:
    for str_chunk in read_chunk(file_obj):
        for char in str_chunk:
            if char in char_frequency:
                char_frequency[char]+=1
            else:
                char_frequency[char]=1

#itemize
char=[]
counts=[]
for item,count in char_frequency.items():
    char.append(item)
    counts.append(count)


In [85]:
"character frequency generation"
import random
print("".join(random.choices(population=char,weights=counts,k=1000)))

fR .redt weeen romt.lyhEng ;r

.o oaezdeIra to,lyw
o,lndnh
uafwhnft henuwgmrIP,SnRss on
r uc eu nea
,oTRheAeieyr. hse Ed i ,s 
s, ucN os t se Athdg
weiretTt ihs.aiold iDtongehsirooiewlfDhe
uhia h, ehle ttirh oSleo. d.e  pafotwtrAl?
aytg’tIeAtmcrefwnI  efo lgSamoooo  lEeeo pdh
Bogbta
togtaS oCiw
EepwehtEIgCslreatrbgomecu seothiliseAeh MSilW uHlr  .arma
dTIh, hbr ,ye adn h natsileiam a  NarWnnrbn  eYhta u M.rslbc oee C a  yIWioaoya s  ’ F_yLee
fwl itc !v’g dAo
b,  —lefrsynmr crg,aro _mw_mtscwsn tn -Usocaoinoaat
oo Ebenn sd
nhoeos r l rruhrekaepbrU
t Us r sheI ee  .Gyiaecoo tT.UahrLwO
l.Bh m n waEtitoeue nhhCbfrmteer igrdltrstrna,Asn ah
x   tr
oe Ahteyi
 l iBa e’fng d
r iitsGsoawOfaaee rolsunolstsc,.Is fu’hhrsr ri rr om de 
 hi r ml rEeAe—
hltsde hmoouaau
s
ooh,le.oelhsspAoef
t i  nenu’  Cvroelgs  m he btt
Ehrieynasslt 
nn t nlf . hhSKset Hrooalss,GiAV m U. br ee
eehubsEctn nf m.?eissTdrinaTe tnlyhaonteayhdeAnwouoM o 
oartfaSone mtn n tcaeshhtiefpeIe ru
elna t
 lnsf saM
w fbJnw 

goe Sph 

In [82]:
from typing import TextIO
import random
def read_chunk(file:TextIO,chunk_size:int=1024)->str:
    while True:
        str_chunk = file.read(chunk_size)
        if not str_chunk:
            break
        yield str_chunk

#now develop some conditional memory
#a small class will help us with book keeping
class MarkovChain():
    EOD="\u0000"
    def __init__(self,window_size:int=1) -> None:
        assert window_size>0
        self._window_size=window_size
        self._memory={}
        self._current_buffer=[str(0x0) for _ in range(window_size+1)] #bootstrap with null chars
        self._generate_buffer=""
        for window in range(window_size+1):
            self._memory[window]={}

    def update(self,character:str)->None:
        assert len(character)==1
        self._current_buffer.append(character)
        self._current_buffer.pop(0)
        for i in range(1,self._window_size+2):
            self.update_item("".join(self._current_buffer[-i:]))


    def update_item(self,string:str)->None:
        window_index=len(string)-1
        if string in self._memory[window_index]:
            self._memory[window_index][string]+=1
        else:
            self._memory[window_index][string]=1

    def finish_training(self)->None:
        """used for end of document signal"""
        self.update(self.EOD)

    def _next_char(self)->str:
        base_count = self._memory[len(self._generate_buffer)-1].get(self._generate_buffer)


        if not base_count:
            raise ValueError("i've never seen this before :(")

        population = []
        counts=[]

        for string,count in self._memory[len(self._generate_buffer)].items():
            if string[0:len(self._generate_buffer)]==self._generate_buffer:
                population.append(string)
                counts.append(count)


        if sum(counts)!=base_count:
            error_msg=f""" COUNTS DON'T MATCH
            buffer={self._generate_buffer}
            count={base_count}
            count_array={counts}, sum={sum(counts)}
            pop={population}
            """
            raise RuntimeError(error_msg)
        return random.choices(population=population,weights=counts,k=1)[0][-1:]

    def generate_string(self,init:str,n=500)->str:
        self._generate_buffer=init[-self._window_size:]

        for _ in range(n):
            next_char = self._next_char()
            if next_char==self.EOD:
                break
            self._generate_buffer = (self._generate_buffer+next_char)[-self._window_size:]
            yield next_char



mc = MarkovChain(window_size=7)

with open("resources/shakespeare.txt", mode="r", encoding="utf-8") as file_obj:
    for str_chunk in read_chunk(file_obj):
        for char in str_chunk:
            mc.update(char)
    mc.finish_training()

In [83]:
init_string = "B"
print(f'{init_string}{"".join([char for char in  mc.generate_string(init_string,1000)])}')


BEROWNE.
I warrant may not an hour,
We are fit you have handsome work, you’d carry out for proof of arms be born, and Timandra.

 Enter Mistress Page for it. Honour.

ANGELO.
My lord is bright than falls like a hidden, hemm’d with you.
Are you well, thou didst intends;
And it grant than wives,
This griefs the hangman of an eye of anything thither—

 Enter Don Pedro is reeling still and waddled all,
The one word.
Boyet, and treble woe!
Come, Margaret was the sighed as my friends that threatens; men that good years,
    And therefore? O me! What, am I constant, lovely said to me, you rogue.

HAMLET.
I am your sight on.

JULIA.
And yet the face by day
Where lock’d in Kent._]

HELENA.
Ay, but bastard and my nipple
Of the wit in the magnanimous and thee
Once more, I beseech you—
Chiefly for a light that was thy wit o’both sides.
Farewell. If thou ever be seen then pardonner aucun prisoner will I be gentle wealth
enough to be one of ’em
I knew you are!
These are thee a hundred and his friend

In [94]:
with open("test.db", mode="r+", encoding="utf-8") as file_obj:
    with mmap.mmap(file_obj.fileno(), length=0, access=mmap.ACCESS_WRITE) as mmap_obj:
        print(int.from_bytes(mmap_obj[500:504],signed=False))
#        mmap_obj[500:504]=0xFFAA.to_bytes(4,signed=False)



65450
